In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df_e = spark.read.format("csv") \
            .option("header", True) \
            .option("inferSchema", True) \
            .load("abfss://raw@netflixprojectdlchetan.dfs.core.windows.net/netflix_titles.csv")


In [0]:
df_e = df_e.fillna({"duration_minutes": "0", "duration_seasons": "1"})

In [0]:
df_e = df_e.withColumn("duration_minutes", col("duration_minutes").cast(IntegerType()))\
           .withColumn("duration_seasons", col("duration_seasons").cast(IntegerType()))

In [0]:
df_e=df_e.withColumn("shorttitle",split(col('title'),':')[0])\
        .withColumn("ratings",split(col('rating'),'-')[0])\
             

In [0]:
from pyspark.sql.functions import when, col; 
df_e=df_e.withColumn("type_flag",when(col('type')=='Movie',1)
                                .when(col('type')=='TV Show',2)\
        .otherwise(0))

In [0]:
from pyspark.sql import Window

In [0]:
df_e.createOrReplaceTempView("temp_view")

In [0]:
df_e = spark.sql('''
                  select * from temp_view
                  ''')


In [0]:
df_e.createOrReplaceGlobalTempView("global_view")

In [0]:
df_e = spark.sql('''
                  select * from global_temp.global_view

                  ''')


In [0]:
df_eb = df_e.groupBy("type").agg(count("*").alias("total_count"))

In [0]:
df_eb.display()

type,total_count
null,1
TV Show,1969
Movie,4265
1944,1


Databricks visualization. Run in Databricks to view.

In [0]:
df_e.write.format("delta")\
    .mode("overwrite")\
        .option("path","abfss://silver@netflixprojectdlchetan.dfs.core.windows.net/netflix_titles")\
            .save()
            